# Facial Recognition Using Python and Open CV

This notebook goes through the process of developing a model that can distinguish between the faces of Barrack Obama and Joe Biden. The main tool we use is the cv2 library, which contains a number of useful functions relating to image processing, face detection, and face recognition. 

I consulted a number of tutorials to figure out how to use the Open CV library to split a video into frames, detect and recognize faces, and rewrite images locally. Some that I found particularly useful are [here](http://eyalarubas.com/face-detection-and-recognition.html), [here](http://cbarker.net/opencv/), and [here](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html). 

First we import the necessary libraries:

In [4]:
import cv2
import os

import numpy as np

from time import time
import logging

import pandas as pd

faceDet = cv2.CascadeClassifier("/Users/Jason/anaconda/pkgs/opencv3-3.1.0-py35_0/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml")
faceDet2 = cv2.CascadeClassifier("/Users/Jason/anaconda/pkgs/opencv3-3.1.0-py35_0/share/OpenCV/haarcascades/haarcascade_frontalface_alt2.xml")
faceDet3 = cv2.CascadeClassifier("/Users/Jason/anaconda/pkgs/opencv3-3.1.0-py35_0/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml")
faceDet4 = cv2.CascadeClassifier("/Users/Jason/anaconda/pkgs/opencv3-3.1.0-py35_0/share/OpenCV/haarcascades/haarcascade_frontalface_alt_tree.xml")

# Train Classifier 

The pictures used here are all frames from various interviews and speeches featuring Obama or Biden. In a separate script, I downloaded videos of them, separated those videos into individual frames using the cv2 library, and used a built in face detector to crop each frame to contain only the relevant face. The code below creates a training set consisting of these frames (represented as 350x350 matrices), and the corresponding labels. 

In [7]:
names = os.listdir('/Users/Jason/Desktop/Machine Learning/Facial Recognition/Pictures for Vid')
del(names[names.index('.DS_Store')])

In [8]:
X, Y, name_vec, n = [], [], [], 0

for name in names:
    files = os.listdir('Final Pics for Vid/%s' %name)
    for file in files:
        if file != ".DS_Store":
            filename = 'Final Pics for Vid/%s' %name + '/' + file
            picture = cv2.imread(filename) # Get each picture 
            gray = cv2.cvtColor(picture, cv2.COLOR_BGR2GRAY) # Convert to grayscale
            X_current = np.asarray(gray, dtype=np.uint8) # Append resulting matrix to training set 
            X.append(X_current) 
            Y.append(n) # Append training labels
            name_vec.append(name)
    n = n + 1
    
    
X = np.asarray(X)
Y = np.asarray(Y)

Here we use cv2's built in LBPH (Local Binary Pattern Histogram) function to train a classifier on the two datasets. This method extracts features by seeing how dark each pixel is compared to its neighbors. It has the advantage of training within a minute or two (vs. 20 + minutes for other methods such as an eigenface classifier), while still performing fairly well. 

In [6]:
recognizer = cv2.face.createLBPHFaceRecognizer()
recognizer.train(X, Y)

## See how the classifier performs on test data

In [5]:
face_cascade = cv2.CascadeClassifier('/Users/Jason/anaconda/pkgs/opencv3-3.1.0-py35_0/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')

In [9]:
images = os.listdir("Video/Frames")
del images[images.index('.DS_Store')]

Here we go through each frame of our test data (a video containing both Obama and Biden). We use cv2's built in face detector to find all faces in the image. Then we predict which person the face belongs to using our model, draw a rectangle around that face, and label it according to our prediction. 

In [10]:
count = 0
for i in range(len(images)):
    image = "frame" + str(i) + ".jpg"
    img = cv2.imread('Video/Frames/%s' %image) # Read in each frame 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to grayscale
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) # Detect all faces
    for (x,y,w,h) in faces: 
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) # Draw rectangle around each face
        roi_gray = gray[y:y+h, x:x+w] # Convert the cropped face to grayscale, predict whose it is w/ classifier
        roi_gray = cv2.resize(roi_gray, (350, 350))
        pred = recognizer.predict(roi_gray)
        if pred == 0:
            name = "Biden"
        else:
            name = "Obama"
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img,name,(x,y + h + 20), font, .5, (255,0,0)) # Write predicted name under image
        num = str(count)
        if len(num) == 1:
            num = "00" + num
        elif len(num) == 2:
            num = "0" + num
        cv2.imwrite("Video/Final_Frames/image%s" %num + '.jpg', img) # Write edited frame out
        
    count += 1  

error: /Users/jenkins/miniconda/1/x64/conda-bld/work/opencv-3.1.0/modules/imgproc/src/color.cpp:7456: error: (-215) scn == 3 || scn == 4 in function ipp_cvtColor


After predicting who the faces in our test data belong to, we can concatenate our labelled frames into a final video. The easiest way to do this is probably with [ffmpeg](http://trac.ffmpeg.org/wiki/Slideshow).  

In [2]:
from IPython.display import HTML

In [3]:
HTML("""
<video width="320" height="240" controls>
  <source src="Video/Final_Frames/LBHP2.mp4" type="video/mp4">
</video>
""")

The resulting video shows that our model worked pretty well. However, there are also some limitations that are evident. 

First of all, Biden's face is not always detected, as it is often not looking directly into the camera. There are also several instances where objects we don't want to recognize are flagged as faces. The most obvious case of this is the statue at the top-left corner of the screen. Additionally, there are a few frames where other objects (a pillar, the folder in Obama's hand, etc...) are flagged as faces. 

This seems to be more of a fault with the face detector than our actual model. One should be aware of the difficulties inherent in classifying faces when they are at irregular angles. However, once a face is detected, our classifier seems to do a pretty good job.  